In [6]:
import json
from pathlib import Path
import time
import datetime
import numpy as np
import pandas as pd
from loguru import logger
import pygmo as pg
from IPython.display import clear_output

from solarmed_optimization import (EnvironmentVariables,
                                   ProblemParameters,
                                   ProblemSamples,
                                   AlgorithmParameters,
                                   PopulationResults)

from solarmed_optimization.utils.initialization import problem_initialization, InitialStates
from solarmed_optimization.utils.evaluation import evaluate_optimization
from solarmed_optimization.utils.serialization import OptimizationResults
from solarmed_optimization.utils.visualization import generate_visualizations
from solarmed_optimization.problems.pygmo import MinlpProblem

# auto reload modules
%load_ext autoreload
%autoreload 2
logger.disable("phd_visualizations")

#%% Constants
# Paths definition
base_output_path: Path = Path("../results")
data_path: Path = Path("../data")
fsm_data_path: Path = Path("../results/fsm_data")
date_str: str = "20180921_20180928" # "20230707_20230710" # '20230630' '20230703'
n_islands: int = 10

if not base_output_path.exists():
    base_output_path.mkdir()

# Either load parameters from json or create a new instance
# with open(output_path / "20230703_eval_at_20250105/gaco/problem_params.json") as f:
#     problem_params = ProblemParameters(**json.load(f))
problem_params: ProblemParameters = ProblemParameters(
    optim_window_time=8*3600, # 8 hours
)
optim_params: AlgorithmParameters = AlgorithmParameters(
    pop_size=16, # 32
    n_gen=3,
    seed_num=32
)
initial_states: InitialStates = InitialStates(Tts_h=[90, 80, 70], 
                                              Tts_c=[70, 60, 50]) 

problem_data = problem_initialization(problem_params=problem_params,
                                      date_str=date_str,
                                      data_path=data_path,
                                      initial_states=initial_states)

ps: ProblemSamples = problem_data.problem_samples
pp: ProblemParameters = problem_data.problem_params
df: pd.DataFrame = problem_data.df
model = problem_data.model

# df_mods: list[pd.DataFrame] = []
df_hors: list[pd.DataFrame] = []
df_sim: pd.DataFrame = None

# Setup optimization algorithm / computation strategy
algorithm = pg.gaco(gen=optim_params.n_gen, ker=optim_params.pop_size, seed=optim_params.seed_num)
algo = pg.algorithm(algorithm)
algo.set_verbosity(1) # regulates both screen and log verbosity

# island = pg.ipyparallel_island()
algo_params =  {
    "gen": optim_params.n_gen, 
    "ker": optim_params.pop_size, 
    "seed": optim_params.seed_num
}
algo_id = "gaco"
metadata: dict[str, str] = {"date_str": date_str, "algo_id": algo_id}
output_path = base_output_path / f"{date_str}_eval_at_{datetime.datetime.now(tz=datetime.timezone.utc):%Y%m%d}" / algo_id
output_path.mkdir(parents=True, exist_ok=True)


2025-01-12 00:10:00.399 | WARNING  | solarmed_optimization.utils.initialization:problem_initialization:100 - External environment data (20220524 - 20240524) (seawater temperature and salinity) is not available for the data period (20180921 - 20180928). Using data for same month and days but from closest year 2022.


In [7]:
output_path.mkdir(parents=True, exist_ok=True)
opt_step_idx: int = 0
max_opt_steps: int = (len(df)-pp.idx_start-ps.optim_window_samples) // ps.n_evals_mod_in_opt_step - 1
idx_mod = pp.idx_start
initial_time = time.time()
opt_step_idx = 0
# for opt_step_idx in range(0, max_opt_steps):
hor_span = (idx_mod+1, idx_mod+1+ps.n_evals_mod_in_hor_window)

# Optimization step `opt_step_idx`
print("")
print(f"Optimization step {opt_step_idx+1}/{max_opt_steps}")

# 1. Initialize the problem instance
## Environment variables predictions
ds = df.iloc[hor_span[0]:hor_span[1]]
env_vars: EnvironmentVariables = EnvironmentVariables(
    I=ds['I'].values,
    Tamb=ds['Tamb'].values,
    Tmed_c_in=ds['Tmed_c_in'].values,
    cost_w=np.ones((ps.n_evals_mod_in_hor_window, )) * pp.env_params.cost_w,
    cost_e=np.ones((ps.n_evals_mod_in_hor_window, )) * pp.env_params.cost_e,
)
## Initialize problem
## Initialize problem
problem = MinlpProblem(
    model=model, 
    sample_time_opt=pp.sample_time_opt,
    optim_window_time=pp.optim_window_time,
    env_vars=env_vars,
    dec_var_updates=pp.dec_var_updates,
    fsm_valid_sequences=pp.fsm_valid_sequences,
    fsm_data_path=fsm_data_path,
    use_inequality_contraints=False
)
prob = pg.problem(problem)

# print(prob)

# Initialize population
# pop = pg.population(prob, size=pop_size, seed=seed_num)
# display(pop)

# Initialize optimization algorithm
# algo = pg.algorithm(pg.gaco(ker=pop_size, gen=100))
# print(f"Running {algo.get_name()}")
# algo.set_verbosity(1) # regulates both screen and log verbosity




Optimization step 1/198


In [14]:
model.sf_ts_state


<SfTsState.HEATING_UP_SF: 1>

In [15]:
from solarmed_modeling.fsms import MedState, SfTsState
from solarmed_optimization.utils.initialization import generate_population
from solarmed_optimization.utils import decision_variables_to_decision_vector
# Initial population for first optimization step
model.med_state = MedState.OFF
model.sf_ts_state = SfTsState.IDLE
pop_dec_vars = generate_population(model=model, pp=pp, problem=problem,
                                   pop_size=optim_params.pop_size, 
                                   prob=prob)

print("Initial population in first optimization step")
for x_idx, x in enumerate(pop_dec_vars):
    print(f"{x_idx:02d}: {x.med_vac_state}, {x.qts_src}")
    
# Imagine that after this step, we are in some other states
# Then, the new initial population should be generated based on this new state
# we use the previous best solution to initialize the real variables, which will
# necesarily match one of the generated integer profiles
best_x = decision_variables_to_decision_vector(pop_dec_vars[1])
model.med_state = MedState.GENERATING_VACUUM
model.sf_ts_state = SfTsState.HEATING_UP_SF

pop_dec_vars = generate_population(model=model, pp=pp, problem=problem,
                                   pop_size=optim_params.pop_size,
                                   dec_vec=best_x)

print("Initial population in first optimization step")
for x_idx, x in enumerate(pop_dec_vars):
    print(f"{x_idx:02d}: {x.med_vac_state}, {x.qts_src}")


Initial population in first optimization step
00: [0 0 0 0 0 0 0 0 0 0], [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
01: [2 2 1 1 1 1 1 1 1 1], [ 0.9  9.8  7.3  4.3 14.9 11.   3.1  6.6 19.9 15.2]
02: [0 2 2 1 1 1 1 1 1 1], [ 0.   9.8  7.3  4.3 14.9 11.   3.1  6.6 19.9 15.2]
03: [0 0 2 2 1 1 1 1 1 1], [ 0.   0.   7.3  4.3 14.9 11.   3.1  6.6 19.9 15.2]
04: [0 0 0 2 2 1 1 1 1 1], [ 0.   0.   0.   4.3 14.9 11.   3.1  6.6 19.9 15.2]
05: [0 0 0 0 2 2 1 1 1 1], [ 0.   0.   0.   0.  14.9 11.   3.1  6.6 19.9 15.2]
06: [0 0 0 0 0 2 2 1 1 1], [ 0.   0.   0.   0.   0.  11.   3.1  6.6 19.9 15.2]
07: [0 0 0 0 0 0 2 2 1 1], [ 0.   0.   0.   0.   0.   0.   3.1  6.6 19.9 15.2]
08: [0 0 0 0 0 0 0 2 2 1], [ 0.   0.   0.   0.   0.   0.   0.   6.6 19.9 15.2]
09: [0 0 0 0 0 0 0 0 2 2], [ 0.   0.   0.   0.   0.   0.   0.   0.  19.9 15.2]
10: [0 0 0 0 0 0 0 0 0 2], [ 0.   0.   0.   0.   0.   0.   0.   0.   0.  15.2]
11: [2 2 1 1 1 1 1 1 1 1], [ 0.9  9.8  7.3  4.3 14.9 11.   3.1  6.6 19.9 15.2]
12: [0 2 2 1 1 1 1 1 1 1],

In [13]:
from solarmed_optimization.problems import validate_real_var_values
from solarmed_optimization.utils import (decision_vector_to_decision_variables,
                                         decision_variables_to_decision_vector)
from solarmed_optimization.utils.initialization import generate_manual_pop

# Manually set initial population. 1st decision vector
pop_dec_vars = generate_manual_pop(model, pp, optim_params.pop_size)
pop = pg.population(prob, size=optim_params.pop_size, seed=optim_params.seed_num) 
for i, dec_vars in enumerate(pop_dec_vars):
    # Initialize random decision vectors
    x = pg.random_decision_vector(prob)
    dec_vars_ = decision_vector_to_decision_variables(x, dec_var_updates=problem.dec_var_updates, span='none', )
    # Set the real decision variables values in the manually generated decision variables
    [setattr(dec_vars, var_id, getattr(dec_vars_, var_id)) for var_id in problem.dec_var_real_ids]
    # Validate the real decision variables values using the manually established logical variables values
    dec_vars = validate_real_var_values(dec_vars, problem.real_dec_vars_box_bounds)
    # pop_dec_vars[i] = dec_vars
    pop.set_x(i, decision_variables_to_decision_vector(dec_vars), )

for x_idx, x in enumerate(pop_dec_vars):
    print(f"{x_idx:02d}: {x.med_vac_state}")
    
# print(pop_dec_vars[1])


00: [0 0 0 0 0 0 0 0 0 0]
01: [2 2 1 1 1 1 1 1 1 1]
02: [0 2 2 1 1 1 1 1 1 1]
03: [0 0 2 2 1 1 1 1 1 1]
04: [0 0 0 2 2 1 1 1 1 1]
05: [0 0 0 0 2 2 1 1 1 1]
06: [0 0 0 0 0 2 2 1 1 1]
07: [0 0 0 0 0 0 2 2 1 1]
08: [0 0 0 0 0 0 0 2 2 1]
09: [0 0 0 0 0 0 0 0 2 2]
10: [0 0 0 0 0 0 0 0 0 2]
11: [2 2 1 1 1 1 1 1 1 1]
12: [0 2 2 1 1 1 1 1 1 1]
13: [0 0 2 2 1 1 1 1 1 1]
14: [0 0 0 2 2 1 1 1 1 1]
15: [0 0 0 0 2 2 1 1 1 1]


In [33]:
from dataclasses import asdict
from solarmed_optimization.problems import DecisionVariables
from solarmed_optimization.utils import resample_decision_variables

new_pop = []
pop_dec_vars_new = [] # Temp for easo of compare with previous population 
# Set initial population from previous step last generation
for x_idx, x in enumerate(pop.get_x()):
    # 1. Convert the decision vectors to decision variables using the optim step sample time
    dec_vars = decision_vector_to_decision_variables(
        x, dec_var_updates=problem.dec_var_updates, 
        span='optim_window', sample_time_mod=pp.sample_time_opt, 
        optim_window_time=pp.optim_window_time
    )
    
    # 2. Take the decision variables from the second element
    # 3. Also duplicate the last element to fill the new value
    dec_vars = DecisionVariables(**{k: np.concatenate((v[1:], [v[-1]] )) for k, v in asdict(dec_vars).items()})

    # 4. Convert back to decision vector making sure each variable has as many values as its number of updates
    # pop.set_x(x_idx, )
    pop_dec_vars_new.append( resample_decision_variables(dec_vars, pp.dec_var_updates) )
    new_pop.append(decision_variables_to_decision_vector(dec_vars, dec_var_updates=pp.dec_var_updates))

for x_idx, x in enumerate(pop_dec_vars_new):
    print(f"{x_idx:02d}: {x.med_vac_state}")


2025-01-11 23:22:37.016 | WARNING  | solarmed_optimization.utils:forward_fill_resample:64 - Source array size 17 is greater than to target size 10. Returning trimmed source array
2025-01-11 23:22:37.019 | WARNING  | solarmed_optimization.utils:forward_fill_resample:64 - Source array size 17 is greater than to target size 10. Returning trimmed source array
2025-01-11 23:22:37.020 | WARNING  | solarmed_optimization.utils:forward_fill_resample:64 - Source array size 17 is greater than to target size 10. Returning trimmed source array
2025-01-11 23:22:37.020 | WARNING  | solarmed_optimization.utils:forward_fill_resample:64 - Source array size 17 is greater than to target size 10. Returning trimmed source array
2025-01-11 23:22:37.021 | WARNING  | solarmed_optimization.utils:forward_fill_resample:64 - Source array size 17 is greater than to target size 10. Returning trimmed source array
2025-01-11 23:22:37.022 | WARNING  | solarmed_optimization.utils:forward_fill_resample:64 - Source array 

00: [0 0 0 0 0 0 0 0 0 0]
01: [2 1 1 1 1 1 1 1 1 1]
02: [2 2 1 1 1 1 1 1 1 1]
03: [0 2 2 1 1 1 1 1 1 1]
04: [0 0 2 2 1 1 1 1 1 1]
05: [0 0 0 2 2 1 1 1 1 1]
06: [0 0 0 0 2 2 1 1 1 1]
07: [0 0 0 0 0 2 2 1 1 1]
08: [0 0 0 0 0 0 2 2 1 1]
09: [0 0 0 0 0 0 0 2 2 2]
10: [0 0 0 0 0 0 0 0 2 2]
11: [2 1 1 1 1 1 1 1 1 1]
12: [2 2 1 1 1 1 1 1 1 1]
13: [0 2 2 1 1 1 1 1 1 1]
14: [0 0 2 2 1 1 1 1 1 1]
15: [0 0 0 2 2 1 1 1 1 1]


In [32]:
pp.dec_var_updates.qts_src


10

In [26]:
for k, v in asdict(dec_vars).items():
    # print(v[1:], v[-1])
    # print(f"{k}: {np.concatenate(v[1:], [v[-1]])}")
    print(v, getattr(pp.dec_var_updates, k))
    downsample_by_segments(np.array(v), getattr(pp.dec_var_updates, k))

# DecisionVariables(**)


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 17


ValueError: target_size must be between 1 and the size of the source array.

In [11]:
getattr(pp.dec_var_updates, "qsf")


17

In [7]:
pop_dec_vars[1]


DecisionVariables(qsf=array([0.4, 4.6, 7.1, 6.3, 3.3, 8. , 0.6, 3.3, 2.9, 2. , 4.7, 8.1, 5.6,
       2.3, 0.7, 6.8, 4.7]), qts_src=array([ 0.9,  4. ,  7.6,  8.2,  9.5, 14.9,  5.4,  0.9,  0.9,  1.7]), qmed_s=array([ 0. ,  0. , 30. , 30. , 30. , 30. , 38.5, 30. , 30. , 30. ]), qmed_f=array([0. , 0. , 5. , 8.4, 5.7, 7.8, 8. , 5. , 5. , 6.3]), Tmed_s_in=array([ 0. ,  0. , 60. , 60. , 60. , 60. , 74.2, 70.6, 60. , 60. ]), Tmed_c_out=array([ 0. ,  0. , 24. , 28.3, 24. , 24. , 24. , 24. , 24. , 24. ]), sf_active=array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True]), ts_active=array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True]), med_active=array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True]), med_vac_state=array([2, 2, 1, 1, 1, 1, 1, 1, 1, 1]))

In [5]:
new_pop[1]


array([ 4.6,  7.1,  6.3,  3.3,  8. ,  0.6,  3.3,  2.9,  2. ,  2. ,  2. ,
        2. ,  2. ,  2. ,  2. ,  2. ,  2. ,  4. ,  7.6,  8.2,  9.5, 14.9,
        5.4,  0.9,  0.9,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,  1.7,
        1.7,  0. , 30. , 30. , 30. , 30. , 38.5, 30. , 30. , 30. , 30. ,
       30. , 30. , 30. , 30. , 30. , 30. , 30. ,  0. ,  5. ,  8.4,  5.7,
        7.8,  8. ,  5. ,  5. ,  6.3,  6.3,  6.3,  6.3,  6.3,  6.3,  6.3,
        6.3,  6.3,  0. , 60. , 60. , 60. , 60. , 74.2, 70.6, 60. , 60. ,
       60. , 60. , 60. , 60. , 60. , 60. , 60. , 60. ,  0. , 24. , 28.3,
       24. , 24. , 24. , 24. , 24. , 24. , 24. , 24. , 24. , 24. , 24. ,
       24. , 24. , 24. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
        1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
        1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
        1. ,  1. ,  1. ,  1. ,  0. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,
        1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1. ,  1

In [ ]:
isl = pg.island(algo = algo,  prob = prob, size = optim_params.pop_size) # udi=island

isl.evolve()
print(isl)

start_time = time.time()
while isl.status == pg.evolve_status.busy:
    time.sleep(5)
    print(f"Elapsed time: {time.time() - start_time:.0f}")
    # print(f"Current evolution results | Best fitness: {pop_current.champion_f[0]}, \nbest decision vector: {pop_current.champion_x}")
optim_eval_elapsed_time = int(time.time() - start_time)
print(f"Completed evolution! Took {time.time() - start_time:.0f} seconds") #| Best fitness: {pop_current.champion_f[0]}, \nbest decision vector: {pop_current.champion_x}")

problem = isl.get_population().problem.extract(object)
# Evaluate optimization and move `model` to the next step
# Update, only evaluate the best in the population, otherwise it takes too much space 
df_hor, df_sim, model = evaluate_optimization(
    df_sim=df_sim, 
    pop=[ isl.get_population().get_x()[isl.get_population().best_idx()] ],
    best_idx=0,
    env_vars=env_vars, problem=problem,
    problem_data=problem_data, idx_mod=idx_mod
)
# if opt_step_idx > 0:
#     df_hors[-1] = df_hors[-1][step_results.best_idx_per_gen[-1]] # Only keep the dataframe from the best individual from past steps
df_hors.append(df_hor[0])

pop_results: PopulationResults = PopulationResults.initialize(
    problem=problem,
    pop_size=optim_params.pop_size,
    n_gen=optim_params.n_gen,
    elapsed_time=optim_eval_elapsed_time,
)

# Export results
OptimizationResults(
    metadata=metadata,
    problem_params=problem_params,
    algo_log=isl.get_algorithm().extract( getattr(pg, algo_id) ).get_log(),
    df_hor=df_hor[0],
    df_sim=df_sim,
    pop_results=pop_results,
    algo_params=algo_params,
    figs=generate_visualizations(problem=problem, df_hors=df_hors, df_sim=df_sim, 
                                 problem_data=problem_data, metadata=metadata, 
                                 pop_results=pop_results)
).dump(output_path=output_path, step_idx=opt_step_idx)

# Finally, increase counter
idx_mod += ps.n_evals_mod_in_opt_step

print(f"Elapsed time: {time.time() - initial_time:.0f}")
print("")

# pop = algo.evolve(pop) 
# print(f"After initiating evolution: \n{isl}")

# uda=algo.extract(pg.gaco)
# print(f"Completed evolution, best fitness: {pop.champion_f[0]}, \nbest decision vector: {pop.champion_x}")

# print(uda.get_log())
# for iter_log in uda.get_log():
#     print(iter_log)

# Archipielago variant
# archi = pg.archipelago(n=n_islands,algo=algo, prob=prob, pop_size=pop_size, seed=seed_num)
# archi.evolve() 
# print(archi)


In [17]:
# Archipielago option
while archi.status == pg.evolve_status.busy:
    clear_output()
    print(archi)
    print(f"Current evolution results per island | Best fitness: {archi.get_champions_f()}, \nbest decision vector: {archi.get_champions_x()}")
    
    time.sleep(5)

print(f"Evolution completed | Best fitness: {archi.get_champions_f()}, \nbest decision vector: {archi.get_champions_x()}")


Evolution completed | Best fitness: [array([-2.3]), array([-2.2]), array([-2.3]), array([-2.2]), array([-2.3]), array([-2.3]), array([-2.2]), array([-2.2]), array([-2.2]), array([-2.2])], 
best decision vector: [array([ 0. ,  0. ,  2.8,  0.6,  8.6,  8.1,  8.5,  2.6,  4.5,  4.6,  7.8,
        8.4,  0. , 18.4, 16.8, 17.2, 12.9, 11.1,  0. ,  0. ,  0. , 16.8,
       41.3, 17.2,  0. ,  0. ,  0. ,  5.5,  5.9,  3.5,  0. ,  0. ,  0. ,
       52.1, 60.7, 61.5,  0. ,  0. ,  0. ,  8.9,  5.4,  2. ,  0. ,  1. ,
        1. ,  0. ,  1. ,  1. ,  0. ,  1. ,  1. ,  1. ,  1. ,  1. ,  0. ,
        0. ,  0. ,  0. ,  1. ,  0. ,  0. ,  1. ,  0. ,  2. ,  2. ,  2. ]), array([ 0. ,  0. ,  1.6,  0.9,  2.3,  4.7,  1.8,  2.5,  5.4,  3.9,  4.1,
        7.7,  0. ,  3.6,  5.2, 18.7,  1.1, 18.7,  0. ,  0. ,  0. , 22.1,
       28. , 38.8,  0. ,  0. ,  0. ,  0.8,  6.5,  1.9,  0. ,  0. ,  0. ,
       12.8, 57.1,  0.7,  0. ,  0. ,  0. , 20.7, 22.1, 26.8,  0. ,  0. ,
        1. ,  0. ,  1. ,  0. ,  0. ,  0. ,  1. ,  1. ,  

In [4]:
# Initialize optimization algorithm
algo = pg.algorithm(pg.gaco(ker=pop_size, gen=10))
print(f"Running {algo.get_name()}")
algo.set_verbosity(1) # regulates both screen and log verbosity

pop = algo.evolve(pop) 

# Extract results of evolution
uda=algo.extract(pg.gaco)
print(f"Completed evolution, best fitness: {pop.champion_f[0]}, \nbest decision vector: {pop.champion_x}")

# print(uda.get_log())
for iter_log in uda.get_log():
    print(iter_log)


Running GACO: Ant Colony Optimization

   Gen:        Fevals:          Best:        Kernel:        Oracle:            dx:            dp:
      1              0      -0.533408              3              0        13.0295     0.00555556
      2              3      -0.552988              3      -0.533408         12.231      0.0240664
      3              6      -0.553859              3      -0.552988        10.9046      0.0211693
      4              9      -0.553859              3      -0.553859        10.9046      0.0210017
      5             12      -0.553859              3      -0.553859        6.82502      0.0165153
      6             15      -0.555301              3      -0.553859        7.93529      0.0179571
      7             18      -0.555301              3      -0.555301        7.93529      0.0176796
      8             21      -0.555631              3      -0.555301        5.84261      0.0180089
      9             24      -0.555631              3      -0.555631        5.62